# Ataque cíclico ao RSA

Ricardo Silva a71532

Francisca Fernandes a72450

João Cerqueira a65432

José Barbosa a69136

## Introdução
Vimos nos trabalhos anteriores como funciona o sistema RSA e o sistema Diffie-Hellman. Vimos também que este último é susceptível a ataques do tipo _man in the middle_, em que uma terceira pessoa colocada entre o destinatário e o remetente consegue decifrar e manipular as mensagens enviadas entre as duas sem que estas se apercebam. Apesar do sistema RSA não sofrer deste problema, não deixa de ser possível decifrar mensagens sem ter acesso à chave privada. Descrevemos neste trabalho como funciona este ataque, conhecido como ataque cíclico ao RSA.


## Envio da mensagem cifrada

Para testar este ataque vamos definir uma chave pública e uma chave privada usando o mesmo algoritmo que definimos no primeiro trabalho.

In [1]:
def RSA_Key(nbits = 512):
    p = random_prime(2^(nbits//2), 2^(nbits//2-1))
    q = random_prime(2^(nbits//2+1), 2^(nbits//2))
    n = p*q
    m = (p-1)*(q-1)
    e = randint(2, m-1)
    while gcd(e, m) != 1:
        e = randint(2, m-1)
    d = power_mod(e, -1, m)
    PubKey = (n, e)
    PrivKey = d
    return PubKey, PrivKey

Geramos então as chaves e guardamos os valores $e$ e $n$ da chave pública.

In [2]:
ChPub, ChPriv = RSA_Key(12)
n = ChPub[0] # n = p*q
e = ChPub[1]
print("Chave publica:", ChPub, "\n")
print("Chave privada:", ChPriv)

Chave publica: (166, 55) 

Chave privada: 3


O remetente gera então a cifra da mensagem através da chave pública e envia para o destinatário.

In [3]:
msg = 123
C = power_mod(msg, e, n)
C

111

## Captura e decifração da mensagem cifrada

No entanto nós capturamos a cifra. Temos então a chave pública $(n,e)$ e a mensagem cifrada $C$. Queremos decifrar a mensagem cifrada $C$ de forma a obter a mensagem original $P$. Para isso, vamos construir a sucessão {$C_{j}$}, com $1\leq C_{j} < n$, em que $C_{1} = C^{e}\ mod\ n$, $C_{2} = C_{1}^{e}\ mod\ n$, ..., $C_{j+1} = C{j}^{e}\ mod\ n$. Se encontrarmos $C_{j} = C$, então sabemos que $C_{j-1} = P$, e temos então a nossa mensagem decifrada.

Interceptamos a mensagem cifrada $C$, e sabemos que $C = m^e\ mod\ n$.
Começamos então por definir $C_{1}$:

In [4]:
C1 = power_mod(C, e, n)
C1

151

$C_{1} \neq C$, pelo que $C_{0} \neq P$ e assim sabemos que para $j = 1$ não deciframos a mensagem. Logo temos de continuar a calcular $C_{2}$, $C_{3}$, ..., $C_{j}$ até encontrarmos um valor que iguale $C$.

Para obter $C_{2}$ calculamos $C_{2} = C_{1}^e\ mod\ n$.

Se $C_{2} \neq C$, então $C_{1} \neq P$. Logo teremos de calcular $C_{3} = C_{2}^e\ mod\ n$.

Facilmente observamos que existe aqui um conjunto de passos cíclicos a executar até que por fim obtemos a nossa mensagem decifrada. Para cada $C_{j} \neq C$, o passo seguinte será calcular $C_{j+1} = C_{j}^e\ mod\ n$, com $1 \leq C_{j} < n$. Este passo será repetido até obtermos $C_{j} = C$, para o qual sabemos então que $C_{j-1} = P$. Obtemos assim a nossa mensagem decifrada.

Podemos então definir um ciclo que calcula $C_{j+1} = C_{j}^e\ mod\ n$  até que encontre o valor de $j$ que nos permita decifrar a mensagem.

In [5]:
Cj = C1
j = 1

while Cj != C:
    j = j + 1
    Cj = power_mod(Cj,e,n) #Cj+1 = Cj^e mod n

print("Cj =", Cj, ", j =", j)

Cj = 111 , j = 8


## Definição do algoritmo

Agora que temos os passos do ataque definidos, podemos escrever os seguintes algoritmos que nos permitem descobrir o valor de $j$ e a mensagem original $P$ a partir da chave pública $(n,e)$ e da mensagem cifrada $C$.

In [6]:
# Este algoritmo indica qual o valor de j de quebra a cifra
def RSACycleAttack_Find_j(n, e, C):
    j = 1
    Cj = power_mod(C, e, n) # Cj = C^e mod n
    while Cj != C:
        j = j + 1
        Cj = power_mod(Cj, e, n) #Cj+1 = Cj^e mod n
    return j

RSACycleAttack_Find_j(n, e, C)

8

In [7]:
# Este algoritmo devolve a mensagem decifrada
def RSACycleAttack(n, e, C):
    j = 1
    C_aux = C
    Cj = power_mod(C, e, n) # Cj = C^e mod n
    while Cj != C:
        j = j + 1
        C_aux = Cj
        Cj = power_mod(Cj, e, n) #Cj+1 = Cj^e mod n
    return C_aux # retorna Cj-1

RSACycleAttack(n,e,C)

123

Como é óbvio, ambos os algoritmos seguem os mesmos passos definidos acima, excepto no retorno das variáveis.

## Como proteger contra o ataque

Apesar de termos conseguido decifrar a mensagem recorrendo a este ataque, apenas obtivemos sucesso devido ao facto de as chaves geradas serem pequenas.

Vamos primeiro calcular o tempo de decifração para a mensagem anterior com o nosso algoritmo:

In [8]:
%%time
RSACycleAttack(n,e,C)

CPU times: user 29 µs, sys: 4 µs, total: 33 µs
Wall time: 35.5 µs


123

Como podemos ver, o ataque foi executado em menos de 1 segundo, pelo que as nossas chaves não são de todo seguras. Tentemos agora com um nível de segurança maior.

Mais uma vez, começamos por gerar as chaves e cifrar a mensagem. Desta vez iremos gerar chaves com 32 bits e usaremos uma mensagem ligeiramente maior.

In [9]:
ChPub, ChPriv = RSA_Key(32)
n = ChPub[0] # n = p*q
e = ChPub[1]
print("Chave publica:", ChPub, "\n")
print("Chave privada:", ChPriv)

# mensagem
msg = 12345678
C = power_mod(msg, e, n)
C

Chave publica: (3440474633, 79923853) 

Chave privada: 2712498949


1552388146

Já temos a nossa mensagem cifrada C, e as chaves pública e privada. Tentemos então aplicar o nosso algorimo para decifar a mensagem.

In [10]:
%%time
RSACycleAttack(n,e,C)

CPU times: user 26.3 s, sys: 1.59 ms, total: 26.3 s
Wall time: 26.3 s


12345678

Podemos ver que o tempo necessário para o ataque funcionar aumentou. Contudo, 32 bits continua a ser um valor baixo para gerar, uma vez que existem chaves com 2048 bits. Vamos então agora testar com chaves de 48 bits.

In [58]:
ChPub, ChPriv = RSA_Key(48)
n = ChPub[0] # n = p*q
e = ChPub[1]
print("Chave publica:", ChPub, "\n")
print("Chave privada:", ChPriv)

# mensagem
msg = 12345678
C = power_mod(msg, e, n)
C

Chave publica: (80428349097599, 1722428388919) 

Chave privada: 77201917425679


32999274747937

Mais uma vez, executamos o ataque sobre a mensagem cifrada:

In [59]:
%%time
RSACycleAttack(n,e,C)

CPU times: user 15min 42s, sys: 32 ms, total: 15min 42s
Wall time: 15min 42s


12345678

Como observado, conseguimos obter a mensagem original $P$, mas o tempo de ataque aumentou de forma exponêncial com o aumento do número de bits para gerar as chaves. 

Fica então claro que, apesar de não ser impossível decifrar uma mensagem recorrendo a este ataque, o custo deste aumenta de forma consideravel à medida que as chaves aumentam de tamanho, tornando-se extremamente caro computar a mensagem original $P$.

Não existe nenhum método especifico de proteção contra o ataque cíclico, visto que todos os valores usados como input do ataque são públicos e o atacante pode ainda tentar usar outro valor de $e$ de forma a tentar facilitar os cálculos (apesar de não existir um valor predefinido $e$ que automaticamente torne o ataque mais rápido).

Mas uma vez que o seu custo é demasiado elevado, a probabilidade de uma mensagem ser  decifrada desta forma é demasiado baixa.